In [ ]:
#data.py

In [ ]:
import idx2numpy
import numpy as np
import os
import pickle



In [ ]:
def load_data(data_directory, train = True):
    if train:
        images = idx2numpy.convert_from_file(os.path.join(data_directory, 'train-images.idx3-ubyte'))
        labels = idx2numpy.convert_from_file(os.path.join(data_directory, 'train-labels.idx1-ubyte'))
    else:
        images = idx2numpy.convert_from_file(os.path.join(data_directory, 't10k-images.idx3-ubyte'))
        labels = idx2numpy.convert_from_file(os.path.join(data_directory, 't10k-labels.idx1-ubyte'))

    vdim = images.shape[1] * images.shape[2]
    vectors = np.empty([images.shape[0], vdim])
    for imnum in range(images.shape[0]):
        imvec = images[imnum, :, :].reshape(vdim, 1).squeeze()
        vectors[imnum, :] = imvec

    return vectors, labels

In [ ]:
train_images, train_labels = load_data('data')

In [ ]:
test_images, test_labels = load_data('data', train = False)

In [ ]:
def z_score_normalize(X, u = None, sd = None):
    """
    Performs z-score normalization on X.

    f(x) = (x - μ) / σ
        where
            μ = mean of x
            σ = standard deviation of x

    Parameters
    ----------
    X : np.array
        The data to z-score normalize
    u (optional) : np.array
        The mean to use when normalizing
    sd (optional) : np.array
        The standard deviation to use when normalizing

    Returns
    -------
        Tuple:
            Transformed dataset with mean 0 and stdev 1
            Computed statistics (mean and stdev) for the dataset to undo z-scoring.
    """

    u = np.mean(X, axis = 1)
    sd = np.std(X, axis = 1)
    X_normalized = np.copy(X)
    for i in range(len(X)):
        X_normalized[i] = (X[i] - u[i])/sd[i]


    return X_normalized, u, sd


In [ ]:
train_normalized, u, sd = z_score_normalize(train_images)

In [ ]:
np.max(train_labels)

9

In [ ]:
def min_max_normalize(X, _min = None, _max = None):
    """
    Performs min-max normalization on X.

    f(x) = (x - min(x)) / (max(x) - min(x))

    Parameters
    ----------
    X : np.array
        The data to min-max normalize
    _min (optional) : np.array
        The min to use when normalizing
    _max (optional) : np.array
        The max to use when normalizing

    Returns
    -------
        Tuple:
            Transformed dataset with all values in [0,1]
            Computed statistics (min and max) for the dataset to undo min-max normalization.
    """
    X_min = np.zeros(len(X))
    X_max = np.zeros(len(X))
    X_normalized = np.zeros((len(X), len(X[0])))

    for i in range(len(X)):
        X_min[i] = np.min(X[i])
        X_max[i] = np.max(X[i])
        X_normalized[i] = (X[i] - X_min[i]) / (X_max[i] - X_min[i])

    return X_normalized, X_min, X_max



In [ ]:
a = np.eye(10)

In [ ]:
a[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
b = np.zeros((3, 7))

In [ ]:
def onehot_encode(y):
    """
    Performs one-hot encoding on y.

    Ideas:
        NumPy's `eye` function

    Parameters
    ----------
    y : np.array
        1d array (length n) of targets (k)

    Returns
    -------
        2d array (shape n*k) with each row corresponding to a one-hot encoded version of the original value.
    """
    diagonals = np.eye(10)
    y_encoded = np.zeros((len(y), 10))

    for i in range(len(y)):
        y_encoded[i] = diagonals[y[i]]

    return y_encoded



In [ ]:
labels_encoded = onehot_encode(train_labels)

In [ ]:
labels_encoded[:10]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [ ]:
labels_encoded[-9:]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [ ]:
def onehot_decode(y):
    """
    Performs one-hot decoding on y.

    Ideas:
        NumPy's `argmax` function

    Parameters
    ----------
    y : np.array
        2d array (shape n*k) with each row corresponding to a one-hot encoded version of the original value.

    Returns
    -------
        1d array (length n) of targets (k)
    """
    y_decoded = np.argmax(y, axis = 1)

    return y_decoded



In [ ]:
labels_decoded = onehot_decode(labels_encoded)

In [ ]:
a = np.array([1,3,5,7,9,11,13,15])

In [ ]:
b = np.random.permutation(8)

In [ ]:
c = a[b]

In [ ]:
c

array([15,  7, 11,  1, 13,  3,  9,  5])

NameError: name 'np' is not defined

In [ ]:
def shuffle(dataset):
    """
    Shuffle dataset.

    Make sure that corresponding images and labels are kept together.
    Ideas:
        NumPy array indexing
            https://numpy.org/doc/stable/user/basics.indexing.html#advanced-indexing

    Parameters
    ----------
    dataset
        Tuple containing
            Images (X)
            Labels (y)

    Returns
    -------
        Tuple containing
            Images (X)
            Labels (y)
    """
    X, y = dataset
    indices = np.random.permutation(len(X))
    X_shuffled = X[indices]
    y_shuffled = y[indices]

    return X_shuffled, y_shuffled



In [ ]:
image_shuffled, label_shuffled = shuffle((train_images, train_labels))

In [ ]:
label_shuffled

array([6, 5, 1, ..., 1, 1, 4], dtype=uint8)

In [ ]:
a = np.array([[3,4], [5,6], [7,8]])

In [ ]:
b = np.insert(a, 0, 1, axis = 1)

In [ ]:
b

array([[1, 3, 4],
       [1, 5, 6],
       [1, 7, 8]])

In [ ]:
def append_bias(X):
    """
    Append bias term for dataset.

    Parameters
    ----------
    X
        2d numpy array with shape (N,d)

    Returns
    -------
        2d numpy array with shape ((N+1),d)
    """
    X_new = np.insert(X, 0, 1, axis = 1)
    return X_new



In [ ]:
train_new = append_bias(train_images)

In [ ]:
train_new.shape

(60000, 785)

In [ ]:
def generate_minibatches(dataset, batch_size=64):
    X, y = dataset
    l_idx, r_idx = 0, batch_size
    while r_idx < len(X):
        yield X[l_idx:r_idx], y[l_idx:r_idx]
        l_idx, r_idx = r_idx, r_idx + batch_size

    yield X[l_idx:], y[l_idx:]

def generate_k_fold_set(dataset, k = 5):
    X, y = dataset
    if k == 1:
        yield (X, y), (X[len(X):], y[len(y):])
        return

    order = np.random.permutation(len(X))

    fold_width = len(X) // k

    l_idx, r_idx = 0, fold_width

    for i in range(k):
        train = np.concatenate([X[order[:l_idx]], X[order[r_idx:]]]), np.concatenate([y[order[:l_idx]], y[order[r_idx:]]])
        validation = X[order[l_idx:r_idx]], y[order[l_idx:r_idx]]
        yield train, validation
        l_idx, r_idx = r_idx, r_idx + fold_width
